In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from joblib import Parallel, delayed
from tqdm import tqdm

In [ ]:
# def get_geoid(lat, lon, counties):
#     # Create a Point object from latitude and longitude
#     point = Point(lon, lat)
    
#     # Ensure the counties shapefile is in the correct projection (usually WGS84, EPSG:4326)
#     counties = counties.to_crs(epsg=4326)
    
#     # Use spatial operations to find the county containing the point
#     matching_county = counties[counties.contains(point)]
    
#     if not matching_county.empty:
#         # Extract the GEOID (replace 'GEOID' with the correct column name if different)
#         geoid = matching_county.iloc[0]['GEOID']
#         return geoid
    
#     return None  # Return None if no match found

# # Function to process each row and extract GEOID
# def process_row(row, counties):
#     lat, lon = row['lat'], row['lon']
#     geoid = get_geoid(lat, lon, counties)
#     return geoid

# def add_geoid_column_parallel(test_csv, counties_shapefile, num_jobs=-1):
#     # Read the counties shapefile
#     counties = gpd.read_file(counties_shapefile)
    
#     # Create a tqdm-enabled list for progress tracking
#     rows = list(test_csv.iterrows())
    
#     # Parallelize the row processing using joblib, wrapping it in tqdm
#     geoid_results = Parallel(n_jobs=num_jobs)(
#         delayed(process_row)(row, counties) for _, row in tqdm(rows, desc="Processing rows", total=len(rows))
#     )
    
#     # Add the GEOID as a new column in the DataFrame
#     test_csv['GEOID'] = geoid_results
    
#     return test_csv

# # Load your test CSV into a DataFrame
# test_csv = pd.read_csv("/data/cher/Sat2Habitat/data/crisp-data-split/test.csv")

# # Path to the shapefile containing county polygons
# counties_shapefile = '/data/cher/Sat2Habitat/data/tl_2023_us_county.zip!tl_2023_us_county'

# # Add county and state columns
# updated_test_csv = add_geoid_column_parallel(test_csv, counties_shapefile)

# # Save the updated DataFrame (optional)
# updated_test_csv.to_csv('updated_test_csv.csv', index=False)

# # View the updated DataFrame with county and state
# print(updated_test_csv.head())


In [19]:
# Attach eco-regions to the data
# # Load your test CSV into a DataFrame
test_csv = pd.read_csv("/data/cher/Sat2Habitat/data/crisp-data-split/test.csv")

geometry = [Point(xy) for xy in zip(test_csv['lon'], test_csv['lat'])]
test_gdf = gpd.GeoDataFrame(test_csv, geometry=geometry, crs="EPSG:4326")

In [20]:
# # ecoregion shapefiles
eco_region3 = gpd.read_file('/data/cher/Sat2Habitat/data/ecoregion/us_eco_l3.zip!us_eco_l3')
eco_region4 = gpd.read_file('/data/cher/Sat2Habitat/data/ecoregion/us_eco_l4_state_boundaries/us_eco_l4.shp')

In [33]:
%%time
eco_region3 = eco_region3.to_crs(test_gdf.crs)
test_gdf_with_ecoregions3 = gpd.sjoin(test_gdf, eco_region3[['NA_L2CODE', 'NA_L2NAME', 'geometry']], how="left")
test_gdf_with_ecoregions3.drop(columns=['index_right'], inplace = True)

eco_region4 = eco_region4.to_crs(test_gdf.crs)
test_gdf_with_ecoregions34 = gpd.sjoin(test_gdf_with_ecoregions3, eco_region4[['NA_L3CODE', 'NA_L3NAME', 'geometry']], how="left")
test_gdf_with_ecoregions34.drop(columns=['geometry','index_right'], inplace = True)

# 


CPU times: user 54 s, sys: 179 μs, total: 54 s
Wall time: 54 s


In [34]:
test_gdf_with_ecoregions34[~test_gdf_with_ecoregions34['NA_L2CODE'].isna()].to_csv('/data/cher/Sat2Habitat/data/crisp-data-split/test_w_eco.csv', index=False)

In [13]:
len(counties['GEOID'].unique())

3235

In [5]:
# only keep images in the test set that exist
import pandas as pd
test = pd.read_csv("/data/cher/Sat2Habitat/data/crisp-data-split/test_imagery.csv")

In [9]:
im_dir = "/data/cher/Sat2Habitat/data/crisp-imagery/bing_test"

# TODO: Remove all image files that are not in the test set
file_paths = [f"/data/cher/Sat2Habitat/data/crisp-imagery/bing_test/{i}.jpg" for i in test['key']]


    

In [18]:
import os

# Directory containing images
im_dir = "/data/cher/Sat2Habitat/data/crisp-imagery/bing_test"

# List of image files that are part of the test set (based on test['key'])
test_keys = test['key'].tolist()  # Assuming 'test' is a DataFrame and 'key' contains the image identifiers

# List all files in the image directory
all_files = os.listdir(im_dir)

# Iterate over the files and remove those not in the test set
for file in all_files:
    # Extract the base name (assuming the file name format is like '123.jpg')
    base_name = os.path.splitext(file)[0]  # Remove the extension
    if int(base_name) not in test_keys:
        # Full path of the file to delete
        file_path = os.path.join(im_dir, file)
        print(f"Removing file: {file_path}")
        os.remove(file_path)  # Remove the file


Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/18128.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/112433.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/18865.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/17905.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/11699.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/186382.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/43581.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/312504.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/211494.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/206622.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/21828.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/17819.jpg
Removing file: /data/cher/Sat2Habitat/data/crisp-imagery/bing_test/2346

In [22]:
test[test['key'] == 274251]

,key,lon,lat
36153,274251,-103.65,30.35


In [17]:
int(base_name) in test_keys

True